In [20]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt

from utils import prepare_data
from build_model import AutoencoderV3
from train_model import train_autoencoder

ImportError: cannot import name 'AutoencoderV3' from 'build_model' (/Users/frankpolak/Desktop/MasterProject/model/build_model.py)

In [6]:
# device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [7]:
# Prepare the data
DNA_meth_train = prepare_data("../Data/DNAMethylation_train.csv")
DNA_meth_test = prepare_data("../Data/DNAMethylation_test.csv")
RNA_seq_train = prepare_data("../Data/RNAseq_train.csv")
RNA_seq_test = prepare_data("../Data/RNAseq_test.csv")

# Merge
X_train = pd.merge(DNA_meth_train, RNA_seq_train, left_index=True, right_index=True)
X_test = pd.merge(DNA_meth_test, RNA_seq_test, left_index=True, right_index=True)

# Convert to tensors and send to device
X_train = torch.tensor(X_train.to_numpy(), dtype=torch.float32).to(device)
X_test = torch.tensor(X_test.to_numpy(), dtype=torch.float32).to(device)

In [ ]:
# 1. Number of Epochs 
model_0 = AutoencoderV3(X_train.shape[1], 250)
model_1 = AutoencoderV3(X_train.shape[1], 125)

# Loss and optimizer
loss_fn = nn.L1Loss()
optimizer_0 = torch.optim.Adam(model_0.parameters(), lr=0.001)
optimizer_1 = torch.optim.Adam(model_1.parameters(), lr=0.001)

# Model training
model_0, train_loss_0, test_loss_0 = train_autoencoder(model=model_0, loss_fn=loss_fn, optimizer=optimizer_0, epochs=1000, X_train=X_train, X_test=X_test, updates=True, updates_epochs=100)
model_1, train_loss_1, test_loss_1 = train_autoencoder(model=model_1, loss_fn=loss_fn, optimizer=optimizer_1, epochs=1000, X_train=X_train, X_test=X_test, updates=True, updates_epochs=100)

# Plot
plt.plot(range(1000), torch.Tensor(train_loss_0).cpu(), color='blue', label="Training loss")
plt.plot(range(1000), torch.Tensor(test_loss_0).cpu(), color='red', label="Test loss")
plt.title("250 hidden units")
plt.legend()
plt.show()

plt.plot(range(1000), torch.Tensor(train_loss_1).cpu(), color='blue', label="Training loss")
plt.plot(range(1000), torch.Tensor(test_loss_1).cpu(), color='red', label="Test loss")
plt.title("125 hidden units")
plt.legend()
plt.show()

(torch.Size([306, 4000]), torch.Size([205, 4000]))

In [17]:
help(train_autoencoder)

NameError: name 'train_autoencoder' is not defined